In [1]:
from glob import glob
import sys
import getpass
import os
# import psutil
from os.path import join, dirname
from os.path import getsize
import pickle
import numpy as np
import pandas as pd
import scipy

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.stats.multitest
# from statsmodels.sandbox.stats.multicomp import multipletests

# import nilearn.masking
from nilearn import plotting as nplt
from nilearn import image as niimg
import nilearn.decoding

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
# from sklearn.svm import LinearSVC

# import mydef as my Class correction for projects
import mydef

/usr/local/anaconda3/envs/sampark/lib/python3.7/site-packages/nilearn/datasets/__init__.py:90: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [2]:
GA = mydef.GA()

Google Drive is detected!


## 1. ROI list

   - Localizer

In [3]:
## ROI
path_list = sorted(glob(join(GA.dir_mask,'localizer','n200_*.nii')))
for path in path_list:
    roi = path.split('/')[-1].replace('_mask.nii', '')
    GA.roi_imgs[roi] = niimg.load_img(path)
GA.roi_imgs

{'n200_c1-1_L_M1': <nibabel.nifti1.Nifti1Image at 0x7fcc806ff290>,
 'n200_c1-2_L_S1': <nibabel.nifti1.Nifti1Image at 0x7fcc7191e210>,
 'n200_c1_L_Postcentral': <nibabel.nifti1.Nifti1Image at 0x7fcc7191e490>,
 'n200_c1_R_SPL': <nibabel.nifti1.Nifti1Image at 0x7fcc7191e6d0>,
 'n200_c2_R_CerebellumIV-V': <nibabel.nifti1.Nifti1Image at 0x7fcc7191e890>,
 'n200_c2_R_MFG': <nibabel.nifti1.Nifti1Image at 0x7fcc7191ead0>,
 'n200_c3_R_MTG': <nibabel.nifti1.Nifti1Image at 0x7fcc7191ed10>,
 'n200_c3_R_Postcentral': <nibabel.nifti1.Nifti1Image at 0x7fcc7191ef10>,
 'n200_c4_L_IFG': <nibabel.nifti1.Nifti1Image at 0x7fcc715841d0>,
 'n200_c4_L_Putamen': <nibabel.nifti1.Nifti1Image at 0x7fcc71584310>,
 'n200_c5_L_SPL': <nibabel.nifti1.Nifti1Image at 0x7fcc71584550>,
 'n200_c5_R_SMA': <nibabel.nifti1.Nifti1Image at 0x7fcc71584750>,
 'n200_c6_R_CerebellumVIIIb': <nibabel.nifti1.Nifti1Image at 0x7fcc71584890>,
 'n200_c6_R_MFG': <nibabel.nifti1.Nifti1Image at 0x7fcc71584b50>,
 'n200_c7_L_Thalamus': <nibabel

In [4]:
key_list = ['n200_c1_L_Postcentral'
            , 'n200_c2_R_CerebellumIV-V'
            , 'n200_c3_R_Postcentral'
            , 'n200_c4_L_Putamen'
            , 'n200_c5_R_SMA'
            , 'n200_c6_R_CerebellumVIIIb'
            , 'n200_c7_L_Thalamus']
temp = {}
for key in key_list:
    temp[key] = GA.roi_imgs[key]
GA.roi_imgs = temp
GA.roi_imgs

{'n200_c1_L_Postcentral': <nibabel.nifti1.Nifti1Image at 0x7fcc7191e490>,
 'n200_c2_R_CerebellumIV-V': <nibabel.nifti1.Nifti1Image at 0x7fcc7191e890>,
 'n200_c3_R_Postcentral': <nibabel.nifti1.Nifti1Image at 0x7fcc7191ef10>,
 'n200_c4_L_Putamen': <nibabel.nifti1.Nifti1Image at 0x7fcc71584310>,
 'n200_c5_R_SMA': <nibabel.nifti1.Nifti1Image at 0x7fcc71584750>,
 'n200_c6_R_CerebellumVIIIb': <nibabel.nifti1.Nifti1Image at 0x7fcc71584890>,
 'n200_c7_L_Thalamus': <nibabel.nifti1.Nifti1Image at 0x7fcc71584d10>}

   - DMN

   - Visual Area

## 2. Visualization ROIs

In [ ]:
GA.draw_rois

## *3. Calculation (run once)

In [5]:
for subj in GA.list_subj:
    for stage in GA.list_stage:
        GA.cross_valid(GA.load_beta(subj, stage), GA.lda)

In [6]:
## save the result as a pickle file
GA.save_as_pkl(data=GA.scores, suffix='7localizers')